In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from mordred import Calculator, descriptors
from rdkit import DataStructs
import numpy as np
import os

from rdkit import RDLogger                                                                                                                                                               
RDLogger.DisableLog('rdApp.*')  

from xgboost import XGBClassifier

from helpers.scorer import *
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, roc_auc_score, auc, roc_curve, accuracy_score, precision_score, recall_score, f1_score, make_scorer, precision_recall_curve, average_precision_score, cohen_kappa_score
from sklearn.model_selection import cross_val_score, cross_validate, RepeatedStratifiedKFold, StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ShuffleSplit, KFold
import math

import deepchem as dc
from rdkit import Chem

# from helpers.scorer import *
from sklearn import preprocessing
import signal, time, random

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/eyal/.local/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'


# My Data + Paper Model

In [2]:
def count_total_chemotypes(toxprint_df,row):
    features = toxprint_df.columns.tolist()
    counter = 0
    for feature in features[1:]:
        counter += row[feature]
    return counter

In [3]:
feature_set1 = ['nHBAcc', 'nHBDon', 'FilterItLogS', 'SLogP', 'bond:C(=O)N_carboxamide_(NH2)', 'bond:C(=O)N_carboxamide_(NHR)',
                'bond:C(=O)N_carboxamide_generic', 'bond:C=O_acyl_hydrazide', 
                'bond:C=O_carbonyl_ab-unsaturated_aliphatic_(michael_acceptors)', 
                'bond:CC(=O)C_ketone_alkene_cyclic_(C6)', 'bond:CC(=O)C_ketone_alkene_cyclic_2-en-1-one', 
                'bond:CN_amine_pri-NH2_aromatic', 'bond:CN_amine_pri-NH2_generic', 'bond:COC_ether_aliphatic',
                'bond:COH_alcohol_diol_(1_3-)', 'bond:CX_halide_aromatic-X_halo_phenol_meta',
                'bond:NC=O_aminocarbonyl_generic', 'bond:NN_hydrazine_acyclic_(connect_noZ)',
                'bond:PC_phosphorus_organo_generic', 'chain:alkaneCyclic_propyl_C3', 'chain:alkyne_ethyne_generic',
                'group:aminoAcid_aminoAcid_generic', 'group:aminoAcid_asparagine', 'group:aminoAcid_leucine',
                'group:carbohydrate_aldopentose', 'group:carbohydrate_ketohexose',
                'group:carbohydrate_pentofuranose_2-deoxy', 'group:carbohydrate_pentofuranose',
                'group:ligand_path_5_bidentate_aminopropanal', 'group:nucleobase_adenine', 'group:nucleobase_uracil',
                'ring:hetero_[5]_N_pyrazole', 'ring:hetero_[5]_O_dioxolane_(1_3-)', 'ring:hetero_[5]_O_furan',
                'ring:hetero_[5]_O_oxolane', 'ring:hetero_[5_6]_N_purine', 'ring:hetero_[6]_N_diazine_(1_3-)_generic',
                'ring:hetero_[6]_N_pyrimidine', 'ring:hetero_[6]_N_pyrimidine_2_4-dione',
                'ring:hetero_[6]_N_triazine_generic', 'ring:hetero_[6]_Z_1_2_4-', 'ring:hetero_[6]_Z_1_3-',
                'ring:hetero_[6]_Z_generic', 'The number of total chemotypes']

In [4]:
toxprint_feat = pd.read_csv('./data/toxprint_V2_new_data_vs_smiles.tsv', sep='\t')
toxprint_feat.shape

(6521, 730)

Here we decide with evaluation to use and with dataset to test on

In [5]:
data_path = './split/db_no_agree_no_dups'

## Helper functions

In [6]:
rdkit_descs = dc.feat.RDKitDescriptors()
mordred = dc.feat.MordredDescriptors(ignore_3D=False)
pubchem = dc.feat.PubChemFingerprint()
mol2vec = dc.feat.Mol2VecFingerprint()
ecfp = dc.feat.CircularFingerprint(size=2048, radius=4)

In [7]:
def calc_descriptors(smiles):
    calc = Calculator([descriptors.SLogP,descriptors.LogS, descriptors.HydrogenBond], ignore_3D=False)
    mordred_desc = [Chem.MolFromSmiles(smi) for smi in smiles]
    # as pandas
    mordred_df = calc.pandas(mordred_desc)
    return mordred_df

In [8]:
def create_svm_data(path):
    df = pd.read_csv(path, usecols=['smiles', 'withdrawn_class'])
    df = pd.merge(df, toxprint_feat, how='inner', left_on='smiles', right_on='M_SMILES')
    
    mordred_df = calc_descriptors(df['smiles'].tolist())
    df = pd.concat([df, mordred_df[['nHBAcc', 'nHBDon', 'FilterItLogS', 'SLogP']]], axis=1)
    
    df['The number of total chemotypes'] = df.apply(lambda x: count_total_chemotypes(toxprint_feat, x), axis=1)

    y = df['withdrawn_class']
    x = df.drop(['smiles', 'withdrawn_class', 'M_SMILES'], axis=1)
    
    return x, y

In [13]:
def create_mol2vec_data(path):
    df = pd.read_csv(path, usecols=['smiles', 'withdrawn_class'])

    features = mol2vec.featurize(df.smiles)
    columns = [f'{str(mol2vec)}_{i}' for i in range(len(features[0]))]
    df = pd.concat([df, pd.DataFrame(features, columns=columns)], axis=1)
        
    y = df['withdrawn_class']
    x = df.drop(['smiles', 'withdrawn_class'], axis=1)
    
    return x, y

In [10]:
def create_all_data(path):
    df = pd.read_csv(path, usecols=['smiles', 'withdrawn_class'])
    df = pd.merge(df, toxprint_feat, how='inner', left_on='smiles', right_on='M_SMILES')
    
    mordred_df = calc_descriptors(df['smiles'].tolist())
    df = pd.concat([df, mordred_df[['nHBAcc', 'nHBDon', 'FilterItLogS', 'SLogP']]], axis=1)
    
    df['The number of total chemotypes'] = df.apply(lambda x: count_total_chemotypes(toxprint_feat, x), axis=1)

    for feat in [rdkit_descs, mol2vec, ecfp]:
        print(f'extracting {feat}')
        features = feat.featurize(df.smiles)
        columns = [f'{str(feat)}_{i}' for i in range(len(features[0]))]
        df = pd.concat([df, pd.DataFrame(features, columns=columns)], axis=1)

        
    y = df['withdrawn_class']
    x = df.drop(['smiles', 'withdrawn_class', 'M_SMILES'], axis=1)
    
    return x, y

# Train TOX FEATURES

In [11]:
for split in ['db_no_agree_no_dups', 'db_agree_no_dups']:
    for dataset in ['ChEMBL', 'DrugBank', 'NCATS']:
        x_train, y_train = create_svm_data(f'./split/{split}/{dataset}/train.csv')
        x_train.columns = [s.replace('[', '_').replace(']', '_') for s in x_train.columns]

        x_test, y_test = create_svm_data(f'./split/{split}/{dataset}/test.csv')
        x_test.columns = [s.replace(',', '_') for s in x_test.columns]
        
        for clf in [SVC(probability=True), XGBClassifier()]:
            clf.fit(x_train, y_train)

            preds = clf.predict_proba(x_test)[:, 1]

            auc = round(roc_auc_score(y_test, preds), 4)
            auc_pr = round(average_precision_score(y_test, preds), 4)

            print(f'{split=} {dataset=} {clf=} AUC={auc}  AUC-PR={auc_pr}')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2565/2565 [00:02<00:00, 1081.45it/s]
/home/eyal/.local/lib/python3.10/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- bond:CX_halide_alkyl-X_bicyclo[2_2_1]heptane
- bond:CX_halide_alkyl-X_bicyclo[2_2_1]heptene
- bond:N=C=O_isocyanate_[O_S]
- bond:N=[N+]=[N-]_azide_aromatic
- bond:N=[N+]=[N-]_azide_generic
- ...
Feature names seen at fit time, yet now missing:
- bond:CX_halide_alkyl-X_bicyclo_2_2_1_heptane
- bond:CX_halide_alkyl-X_bicyclo_2_2_1_heptene
- bond:N=C=O_isocyanate__O_S_
- bond:N=_N+_=_N-__azide_aromatic
- bond:N=_N+_=_N-__azide_generic
- ...

  warnings.warn(message, FutureWarning)


split='db_no_agree_no_dups' dataset='ChEMBL' clf=SVC(probability=True) AUC=0.5571  AUC-PR=0.2733
split='db_no_agree_no_dups' dataset='ChEMBL' clf=XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...) AUC=0.5542  AUC-PR=0.2671


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2431/2431 [00:02<00:00, 1125.35it/s]
/home/eyal/.local/lib/python3.10/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- bond:CX_halide_alkyl-X_bicyclo[2_2_1]heptane
- bond:CX_halide_alkyl-X_bicyclo[2_2_1]heptene
- bond:N=C=O_isocyanate_[O_S]
- bond:N=[N+]=[N-]_azide_aromatic
- bond:N=[N+]=[N-]_azide_generic
- ...
Feature names seen at fit time, yet now missing:
- bond:CX_halide_alkyl-X_bicyclo_2_2_1_heptane
- bond:CX_halide_alkyl-X_bicyclo_2_2_1_heptene
- bond:N=C=O_isocyanate__O_S_
- bond:N=_N+_=_N-__azide_aromatic
- bond:N=_N+_=_N-__azide_generic
- ...

  warnings.warn(message, FutureWarning)


split='db_no_agree_no_dups' dataset='DrugBank' clf=SVC(probability=True) AUC=0.6035  AUC-PR=0.123
split='db_no_agree_no_dups' dataset='DrugBank' clf=XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...) AUC=0.5606  AUC-PR=0.1031


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4013/4013 [00:03<00:00, 1151.59it/s]
/home/eyal/.local/lib/python3.10/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- bond:CX_halide_alkyl-X_bicyclo[2_2_1]heptane
- bond:CX_halide_alkyl-X_bicyclo[2_2_1]heptene
- bond:N=C=O_isocyanate_[O_S]
- bond:N=[N+]=[N-]_azide_aromatic
- bond:N=[N+]=[N-]_azide_generic
- ...
Feature names seen at fit time, yet now missing:
- bond:CX_halide_alkyl-X_bicyclo_2_2_1_heptane
- bond:CX_halide_alkyl-X_bicyclo_2_2_1_heptene
- bond:N=C=O_isocyanate__O_S_
- bond:N=_N+_=_N-__azide_aromatic
- bond:N=_N+_=_N-__azide_generic
- ...

  warnings.warn(message, FutureWarning)


...

split='db_agree_no_dups' dataset='ChEMBL' clf=SVC(probability=True) AUC=0.6033  AUC-PR=0.228
split='db_agree_no_dups' dataset='ChEMBL' clf=XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...) AUC=0.6362  AUC-PR=0.2642


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1982/1982 [00:01<00:00, 1045.45it/s]
/home/eyal/.local/lib/python3.10/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- bond:CX_halide_alkyl-X_bicyclo[2_2_1]heptane
- bond:CX_halide_alkyl-X_bicyclo[2_2_1]heptene
- bond:N=C=O_isocyanate_[O_S]
- bond:N=[N+]=[N-]_azide_aromatic
- bond:N=[N+]=[N-]_azide_generic
- ...
Feature names seen at fit time, yet now missing:
- bond:CX_halide_alkyl-X_bicyclo_2_2_1_heptane
- bond:CX_halide_alkyl-X_bicyclo_2_2_1_heptene
- bond:N=C=O_isocyanate__O_S_
- bond:N=_N+_=_N-__azide_aromatic
- bond:N=_N+_=_N-__azide_generic
- ...

  warnings.warn(message, FutureWarning)


split='db_agree_no_dups' dataset='DrugBank' clf=SVC(probability=True) AUC=0.6499  AUC-PR=0.1466
split='db_agree_no_dups' dataset='DrugBank' clf=XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...) AUC=0.674  AUC-PR=0.1473


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3601/3601 [00:03<00:00, 1090.31it/s]
/home/eyal/.local/lib/python3.10/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- bond:CX_halide_alkyl-X_bicyclo[2_2_1]heptane
- bond:CX_halide_alkyl-X_bicyclo[2_2_1]heptene
- bond:N=C=O_isocyanate_[O_S]
- bond:N=[N+]=[N-]_azide_aromatic
- bond:N=[N+]=[N-]_azide_generic
- ...
Feature names seen at fit time, yet now missing:
- bond:CX_halide_alkyl-X_bicyclo_2_2_1_heptane
- bond:CX_halide_alkyl-X_bicyclo_2_2_1_heptene
- bond:N=C=O_isocyanate__O_S_
- bond:N=_N+_=_N-__azide_aromatic
- bond:N=_N+_=_N-__azide_generic
- ...

  warnings.warn(message, FutureWarning)


split='db_agree_no_dups' dataset='NCATS' clf=SVC(probability=True) AUC=0.5479  AUC-PR=0.2594
split='db_agree_no_dups' dataset='NCATS' clf=XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...) AUC=0.5466  AUC-PR=0.263


# Train MOL2VEC Features

In [14]:
for split in ['db_no_agree_no_dups', 'db_agree_no_dups']:
    for dataset in ['ChEMBL', 'DrugBank', 'NCATS']:
        x_train, y_train = create_mol2vec_data(f'./split/{split}/{dataset}/train.csv')
        x_train.columns = [s.replace('[', '_').replace(']', '_') for s in x_train.columns]

        x_test, y_test = create_mol2vec_data(f'./split/{split}/{dataset}/test.csv')
        x_test.columns = [s.replace(',', '_') for s in x_test.columns]
        
        for clf in [SVC(probability=True), XGBClassifier()]:
            clf.fit(x_train, y_train)

            preds = clf.predict_proba(x_test)[:, 1]

            auc = round(roc_auc_score(y_test, preds), 4)
            auc_pr = round(average_precision_score(y_test, preds), 4)

            print(f'{split=} {dataset=} {clf=} AUC={auc}  AUC-PR={auc_pr}')

split='db_no_agree_no_dups' dataset='ChEMBL' clf=SVC(probability=True) AUC=0.5952  AUC-PR=0.3031
split='db_no_agree_no_dups' dataset='ChEMBL' clf=XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...) AUC=0.4679  AUC-PR=0.2179
split='db_no_agree_no_dups' dataset='DrugBank' clf=SVC(probability=True) AUC=0.6013  AUC-PR=0.1132
split='db_no_agree_no_dups' da

# Train with All Features

In [ ]:
for split in ['db_no_agree_no_dups', 'db_agree_no_dups']:
    for dataset in ['ChEMBL', 'DrugBank', 'NCATS']:
        x_train, y_train = create_all_data(f'./split/{split}/{dataset}/train.csv')
        x_train.columns = [s.replace('[', '_').replace(']', '_') for s in x_train.columns]
#         x_train.fillna(0, inplace=True)

        x_test, y_test = create_all_data(f'./split/{split}/{dataset}/test.csv')
        x_test.columns = [s.replace(',', '_') for s in x_test.columns]
#         x_test.fillna(0, inplace=True)
        
        for clf in [XGBClassifier()]:
            clf.fit(x_train, y_train)

            preds = clf.predict_proba(x_test)[:, 1]

            auc = round(roc_auc_score(y_test, preds), 4)
            auc_pr = round(average_precision_score(y_test, preds), 4)

            print(f'{split=} {dataset=} {clf=} AUC={auc}  AUC-PR={auc_pr}')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3779/3779 [00:03<00:00, 1088.79it/s]


extracting RDKitDescriptors
extracting Mol2VecFingerprint
extracting CircularFingerprint_radius_4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2565/2565 [00:02<00:00, 1094.29it/s]


extracting RDKitDescriptors
extracting Mol2VecFingerprint
extracting CircularFingerprint_radius_4
split='db_no_agree_no_dups' dataset='ChEMBL' clf=XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...) AUC=0.4732  AUC-PR=0.225


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3998/3998 [00:03<00:00, 1062.93it/s]


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2354/2354 [00:02<00:00, 1043.52it/s]


extracting RDKitDescriptors
extracting Mol2VecFingerprint
extracting CircularFingerprint_radius_4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4013/4013 [00:03<00:00, 1089.19it/s]


extracting RDKitDescriptors
extracting Mol2VecFingerprint
extracting CircularFingerprint_radius_4
split='db_no_agree_no_dups' dataset='NCATS' clf=XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...) AUC=0.4869  AUC-PR=0.2581


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3779/3779 [00:03<00:00, 1094.02it/s]
